In [47]:
import requests
import time
import json
import os


In [48]:
def get_us_election_titles():
    """
    Returns lists of Wikipedia article titles for US elections in 2016, 2020, and 2024.
    """
    elections_2016 = [
        "2016_United_States_presidential_election",
        "2016_Democratic_Party_presidential_primaries",
        "2016_Republican_Party_presidential_primaries"
    ]
    
    elections_2020 = [
        "2020_United_States_presidential_election",
        "2020_Democratic_Party_presidential_primaries",
        "2020_Republican_Party_presidential_primaries"
    
    elections_2024 = [
        "2024_United_States_presidential_election",
        "2024_Democratic_Party_presidential_primaries",
        "2024_Republican_Party_presidential_primaries"
    ]
    
    return {
        "2016": elections_2016,
        "2020": elections_2020,
        "2024": elections_2024
    }


In [49]:
def fetch_articles_in_batches(titles_batch):
    """
    Fetches up to 50 articles in a single API call.
    """
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "prop": "revisions",
        "rvslots": "*",
        "rvprop": "content",
        "format": "json",
        "titles": "|".join(titles_batch)  # Batch up to 50 titles
    }
    headers = {"User-Agent": "WikiSandbox/ManagingBigData"}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"Error fetching articles: {response.status_code}")
    return response.json()

In [50]:
def fetch_article(title):
    """
    Fetches a single article by title.
    """
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "prop": "revisions",
        "rvslots": "*",
        "rvprop": "content",
        "format": "json",
        "titles": title
    }
    headers = {"User-Agent": "WikiSandbox/ManagingBigData"}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"Error fetching article {title}: {response.status_code}")
    return response.json()

In [51]:
def save_to_hdfs(data, file_path):
    """
    Saves the data to a file formatted for HDFS ingestion.
    """
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, "a", encoding="utf-8") as f:
        for article in data:
            json.dump(article, f, ensure_ascii=False)
            f.write("\n")  # Write each article as a separate JSON line

In [52]:
def fetch_and_save_articles(titles, output_file, batch_mode=True):
    """
    Fetches full text for a list of articles and saves them in an HDFS-compatible format.

    Args:
        titles (list): List of article titles to fetch.
        output_file (str): Path to the output file.
        batch_mode (bool): Whether to use batch fetching or individual fetching.
    """
    total_saved = 0  # Track the total number of articles saved
    batch_size = 50  # Max 50 titles per batch

    if batch_mode:
        # Process in batches
        batches = [titles[i:i + batch_size] for i in range(0, len(titles), batch_size)]
        for idx, batch in enumerate(batches, start=1):
            print(f"Fetching batch {idx}/{len(batches)}...")
            try:
                data = fetch_articles_in_batches(batch)
                processed_data = []
                for page_id, page_info in data["query"]["pages"].items():
                    title = page_info["title"]
                    if "revisions" in page_info:
                        text = page_info["revisions"][0]["slots"]["main"]["*"]
                    else:
                        text = "No content available"
                    processed_data.append({"title": title, "text": text})
                save_to_hdfs(processed_data, output_file)
                total_saved += len(processed_data)
            except Exception as e:
                print(f"Failed to fetch batch {idx}: {e}")
            time.sleep(1)  # Wait 1 second between batches
    else:
        # Process individually
        for idx, title in enumerate(titles, start=1):
            print(f"Fetching article {idx}/{len(titles)}: {title}...")
            try:
                data = fetch_article(title)
                processed_data = []
                for page_id, page_info in data["query"]["pages"].items():
                    if "revisions" in page_info:
                        text = page_info["revisions"][0]["slots"]["main"]["*"]
                    else:
                        text = "No content available"
                    processed_data.append({"title": page_info["title"], "text": text})
                save_to_hdfs(processed_data, output_file)
                total_saved += 1
            except Exception as e:
                print(f"Failed to fetch article {title}: {e}")
            time.sleep(1)  # Wait 1 second between requests

    print(f"Total articles saved: {total_saved}")

In [53]:
# Get US election titles
election_titles = get_us_election_titles()
batch_mode = False  # Set to False to disable batch fetching

# Process each election year
for year, titles in election_titles.items():
    output_file = f"hdfs_data/us_elections_{year}.jsonl"
    print(f"Fetching articles for US Elections {year}...")
    fetch_and_save_articles(titles, output_file, batch_mode=batch_mode)

Fetching articles for US Elections 2016...
Fetching article 1/3: 2016_United_States_presidential_election...
Fetching article 2/3: 2016_Democratic_Party_presidential_primaries...
Fetching article 3/3: 2016_Republican_Party_presidential_primaries...
Total articles saved: 3
Fetching articles for US Elections 2020...
Fetching article 1/1: 2020_United_States_presidential_election...
Total articles saved: 1
Fetching articles for US Elections 2024...
Fetching article 1/1: 2024_United_States_presidential_election...
Total articles saved: 1
